In [0]:
CREATE EXTERNAL LOCATION dea-course-ext-dl-gizmobox
URL 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/'
WITH (STORAGE CREDENTIAL dea-course-ext-sc)

In [0]:
%fs ls 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/landing/external-data/'

In [0]:
CREATE CATALOG IF NOT EXISTS gizmobox
MANAGED LOCATION 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/'
COMMENT 'This is a gizmo-box catalog'

In [0]:
use catalog gizmobox

In [0]:
create schema if not exists landing
comment 'landing zone'
managed location 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/landing';

create schema if not exists bronze
comment 'bronze zone'
managed location 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/bronze';

create schema if not exists silver
comment 'silver zone'
managed location 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/silver';

create schema if not exists gold
comment 'gold zone'
managed location 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/gold';


In [0]:
show schemas

In [0]:
use catalog gizmobox;
use schema landing;

create external volume if not exists operational_data
location 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/landing/operational-data/'

In [0]:
select
_metadata.file_name,
_metadata.file_path,
* 
from json.`/Volumes/gizmobox/landing/operational_data/customers`

In [0]:
use catalog gizmobox;
use schema bronze;

CREATE OR REPLACE VIEW v_customers AS
select
_metadata.file_name,
_metadata.file_path,
* 
from json.`/Volumes/gizmobox/landing/operational_data/customers`

In [0]:
select *
from gizmobox.bronze.v_customers
where customer_id > 5000;

In [0]:
select *
from text.`/Volumes/gizmobox/landing/operational_data/orders`

In [0]:
use catalog gizmobox;
use schema bronze;
CREATE OR REPLACE VIEW gizmobox.bronze.v_orders AS
select *
from text.`/Volumes/gizmobox/landing/operational_data/orders`

In [0]:
select *
from v_orders;

In [0]:
use catalog gizmobox;
use schema bronze;
create or replace table gizmobox.bronze.v_memberships
as
select *
from binaryFile.`/Volumes/gizmobox/landing/operational_data/memberships/*/*.png`


In [0]:
select *
from v_memberships

In [0]:
select * from csv.`/Volumes/gizmobox/landing/operational_data/addresses`

In [0]:
use catalog gizmobox;
use schema bronze;

create or replace view gizmobox.bronze.v_addresses
as

select *
from read_files('/Volumes/gizmobox/landing/operational_data/addresses',
                format=>'csv',
                delimiter=>'\t',
                header=>true)


In [0]:
select *
from v_addresses

In [0]:
%fs ls 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/landing/external-data/payments'



In [0]:
CREATE TABLE IF NOT EXISTS gizmobox.bronze.payments
(payment_id INTEGER,
order_id INTEGER,
payment_timestamp TIMESTAMP,
payment_status INTEGER,
payment_method STRING)
USING csv
OPTIONS (header = 'true',
         delimeter = ','
)
LOCATION 'abfss://gizmo-box@deacourseextdlmarek.dfs.core.windows.net/landing/external-data/payments'

In [0]:
select * from payments

In [0]:
select * from payments

In [0]:
use catalog hive_metastore;
use schema bronze;

create table if not exists refunds
Using jdbc
OPTIONS (
  url 'jdbc:sqlserver://gizmobox-srv-marek.database.windows.net:1433;database=gizmobox-db',
  dbtable 'refunds',
  user 'gizmoboxadm',
  password '@Xswedc23@');



In [0]:
select * from refunds;